In [4]:
import os
import json
from MilvusClient import MilvusClient
milvus_client = MilvusClient()

In [5]:
# Create a collection with the job positions if it doesn't exist
# Note you can do this also by running the job_positions_ingestion.py script
# This is just for free-play

collection_name = os.getenv("MILVUS_COLLECTION")

if milvus_client.client.has_collection(collection_name):
    print(f"Collection {collection_name} already exists. Skipping.")
else:
    print(f"Collection {collection_name} does not exist. Creating...")
    job_positions_file_path = "job_positions.json"
    with open(job_positions_file_path, 'r') as f:
        job_positions = json.load(f)
    result = milvus_client.create_and_load_collection(collection_name, job_positions)


Collection job_positions does not exist. Creating...
2025-04-12 22:39:08,648 - DEBUG - milvus_logger - Created collection: job_positions
2025-04-12 22:39:13,654 - DEBUG - milvus_logger - Prepared 10 documents with embeddings
2025-04-12 22:39:13,675 - DEBUG - milvus_logger - Inserted 10 documents into job_positions


In [7]:
# You can play with Milvus vector database here to check job positions
# This job positions will be retrieved by the RAG agent
test_query = """
Unity, Unreal Engine, C#, C++, Python, Git, Perforce, Jira, Visual Studio, Blender, Maya, Photoshop, Gameplay Mechanics, AI Programming, UI/UX Implementation, Physics Simulation, Performance Optimization, Shader Development, Agile, Scrum, Waterfall, Game Design Principles, 3D Mathematics"""

print("=" * 50)
print(f"\nQuerying with skills: {test_query}")
print("=" * 50)

results = milvus_client.hybrid_search(
    collection_name=collection_name,
    query_text=test_query,
    limit=3,
    ranker_type="weighted",
    sparse_weight=0.3,
    dense_weight=0.7,
    output_fields=["job_id", "title", "company_name", "required_skills"]
)

print(f"Found {len(results)} matches:")
for i, hit in enumerate(results):
    print(f"  {i}. {hit['entity']['title']} at {hit['entity']['company_name']}")
    print(f"     Job ID: {hit['entity']['job_id']}")
    print(f"     Match Score: {hit['distance']:.4f}")
    print(f"     Required Skills: {hit['entity']['required_skills']}")
    print()


Querying with skills: 
Unity, Unreal Engine, C#, C++, Python, Git, Perforce, Jira, Visual Studio, Blender, Maya, Photoshop, Gameplay Mechanics, AI Programming, UI/UX Implementation, Physics Simulation, Performance Optimization, Shader Development, Agile, Scrum, Waterfall, Game Design Principles, 3D Mathematics
2025-04-12 22:39:34,384 - DEBUG - milvus_logger - Using WeightedRanker with weights 0.3 and 0.7
Found 3 matches:
  0. Unity Game Developer at GameVerse Studios
     Job ID: GAME-DEV-007
     Match Score: 0.7953
     Required Skills: Unity, C#, Game Design, 3D Modeling, Animation, Physics, AR/VR, Git, Agile, Problem-solving

  1. Full Stack Developer at WebMatrix Technologies
     Job ID: DEV-FS-002
     Match Score: 0.7171
     Required Skills: JavaScript, TypeScript, React, Node.js, Express, MongoDB, REST APIs, Git, Agile, CI/CD

  2. Machine Learning Engineer at TechInnovate Solutions
     Job ID: ML-DS-001
     Match Score: 0.5042
     Required Skills: Python, TensorFlow, PyT